In [ ]:
import requests
import json
def query_llm_rest_pix(mlist,image=None):
        
        url = 'http://<IP>:9109/v1/chat/completions'
        headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer token'
        }
        data = {
            "model": "mistralai/Pixtral-12B-2409"
        }
        data.update({"messages": mlist})

        response = requests.post(url, headers=headers, data=json.dumps(data))
        # print(response.status_code)
        # print(response)
        if response.status_code == 400:
            return "I'm unsure about the answer. Please provide more context or ask a different question."
        else:
            return response.json()["choices"][0]['message']['content']

# prm = """



# example response:
# ['keyword1', 'keyword2', 'keyword3']"""

# response = query_llm_rest_pix([{"role": "user", "content": [{"type": "text", "text": prm}]}])


In [8]:
response

"['MODULE', '3', 'Practice', 'Life', 'Insurance', 'Notes', 'Claims', 'and', 'Settlement', 'DIPLOMA', 'INSURANCE', 'SERVICES', 'INTRODUCTION', 'OBJECTIVES', 'CLAIM', 'SETTLEMENT', 'Payment', 'claim', 'life', 'insurance', 'premium', 'Survival', 'Benefit', 'Claim', 'Maturity', 'Benfit', 'Claim', 'Death', 'Benefit', 'Claim', 'procedure', 'documents', 'forms', 'Insurance', 'policy', 'original', 'policy', 'document', 'age', 'proof', 'intimation', 'death', 'nominee', 'received', 'claim', 'nomination', 'policy', 'life', 'assured', 'disappeared', 'premature', 'death', 'claim', 'Indian', 'Evidence', 'Act', 'probate', 'succession', 'certificate', 'administrator', 'general', 'death', 'occurred', 'claim', 'concession', 'ex', 'gratia', 'claim', 'Ex', 'gratia', 'claims', 'options', 'IRDA', 'REGULATION', 'POLICYHOLDERS', '', 'PROCEDURE', 'CLAIM', 'Settlement', 'regulations', 'policyholder', 'interests', 'claim', 'paid', 'reasons', 'relevant', 'papers', 'interest', 'bank', 'rate', 'SUMMARY', 'terminal'

In [ ]:
import os
import json
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import TokenTextSplitter

import requests
import json


import time
from chromadb import HttpClient
from chromadb.utils import embedding_functions

chroma_client = HttpClient(host="localhost", port=9158)


from PyPDF2 import PdfReader
from typing import List
from langchain.text_splitter import RecursiveCharacterTextSplitter

def query_llm_rest_pix(mlist,image=None):
        
        url = 'http://172.19.104.12:9109/v1/chat/completions'
        headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer token'
        }
        data = {
            "model": "mistralai/Pixtral-12B-2409"
        }
        data.update({"messages": mlist})

        response = requests.post(url, headers=headers, data=json.dumps(data))
        # print(response.status_code)
        # print(response)
        if response.status_code == 400:
            return "I'm unsure about the answer. Please provide more context or ask a different question."
        else:
            return response.json()["choices"][0]['message']['content']



def extract_text_from_pdf(pdf_path: str) -> List[str]:
    reader = PdfReader(pdf_path)
    chunks = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

    for page in reader.pages:
        text = page.extract_text()
        paragraphs = text.split('\n\n')
        for paragraph in paragraphs:
            if paragraph.strip():
                chunks.extend(splitter.split_text(paragraph.strip()))
    print("chunks--------------------------------------\n",len(chunks))
    return chunks


extract_keywords = False
# Function to load, chunk, and store PDFs into vector stores
def chunk_and_store(pdf_path: str, db_name: str,):
    # Load PDF
    documents = []
    print("pdf:", pdf_path)
    loader = PyPDFLoader(pdf_path)
    documents.extend(loader.load())
    text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=0)

    chunks = text_splitter.split_documents(documents)
    combined_chunks = " ".join([i.page_content for i in chunks])


    # text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=0)

    # docs = text_splitter.split_documents(documents)
    collection = chroma_client.get_or_create_collection(db_name,embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2"))

    collection.add(ids=[f"doc_{i}" for i in range(len(chunks))], documents=[i.page_content for i in chunks],metadatas=[i.metadata for i in chunks])

    print(f"Stored {len(chunks)} chunks in vector store.")
    print("combined_chunks--------------------------------------\n",combined_chunks)
    # Extract keywords from chunks using LLM
    # combined_chunks = " ".join(chunks)
    if extract_keywords:
        # print(chunk)
        prompt = f"""Extract only the keywords from the following text. Return response in a python list. Do not include any other text in the response:\n\n
                {combined_chunks}\n\n
                example response:
                ['keyword1', 'keyword2', 'keyword3']
                """
        # print(prompt)
        
        response = query_llm_rest_pix([{"role": "user", "content": [{"type": "text", "text": prompt}]}])
        print(response)
        import re
        import ast   
        match = re.search(r"\[.*\]", response, re.DOTALL)

        if match:
            keyword_list = ast.literal_eval(match.group(0))  # Safely convert to Python list
            print(keyword_list)
        else:
            keyword_list = []
            print("No list found in response.")
        # Save keywords to a dictionary
        keywords_dict = {}
        keywords_dict[db_name] = keyword_list

        # Open the file and update the dictionary
        keywords_file = "/home/vikasnr/codebase/crsl/scratch/new_keywords.json"
        if os.path.exists(keywords_file):
            with open(keywords_file, "r") as f:
                existing_keywords = json.load(f)
        else:
            existing_keywords = {}

        existing_keywords.update(keywords_dict)

        with open(keywords_file, "w") as f:
            json.dump(existing_keywords, f, indent=4)

# # Function to process PDF files and store embeddings
# def process_pdf_files(pdf_file: str, vector_store: Chroma):
#     # Load PDF files

#     loader = PyMuPDFLoader(pdf_file)
#     documents = loader.load()
   
#     # Split text recursively
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#     split_docs = text_splitter.split_documents(documents)
   
#     # Store embeddings
#     vector_store.add_documents(split_docs)


pdf1 = "insurance"
pdf2 = "rivers"
pdf3 = "wildlife"


# Load PDFs
pdf_files = {pdf1: f"/home/vikasnr/codebase/crsl/pdfs/{pdf1}.pdf", 
             pdf2: f"/home/vikasnr/codebase/crsl/pdfs/{pdf2}.pdf", 
             pdf3: f"/home/vikasnr/codebase/crsl/pdfs/{pdf3}.pdf"}

for db_name, pdf_path in pdf_files.items():
    chunk_and_store(pdf_path, db_name)



pdf: /home/vikasnr/codebase/crsl/pdfs/insurance.pdf
Stored 17 chunks in vector store.
combined_chunks--------------------------------------
 MODULE - 3
Practice of Life Insurance
Notes
 103
Claims and Settlement
DIPLOMA IN INSURANCE SERVICES
7.0 INTRODUCTION
The Insurance Policy is taken by the consumers to compensate
them in the event of happening of an unforeseen event. It is a
hedge against unavoidable circumstances. In general
insurance the loss is payable only on happening of some
specific event. If the insured does not suffer any loss no claim
is paid to him. The premium is charged on yearly basis and
no accumulation takes place. However the scenario is different
in case of life insurance. If the insured dies during the policy
period he gets the sum assured along with the bonus accrued
under the policy if any. If the insured survives the policy period
he gets the maturity amount accrued under the policy. In this
lesson we shall learn the various aspects in settlement of life
insu

In [ ]:
# List all documents in vector_store_1
collection = chroma_client.get_or_create_collection(pdf2,embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2"))
results = collection.query(query_texts=["What are the function of rivers"], n_results=5, include=["documents"])
results["documents"]
# retrieved_texts = [doc["text"] for doc in results]



[["engineering structures such as canals, pipes, or culverts. Rivers make up only a tiny \npercentage of the world's water. Ninety seven percent of the world's water is held in the oceans, of the remaining three percent which is  fresh water, three quarters is stored as \nland ice (i.e., in glaciers, ice caps), lakes c ontain half a percent, soil moisture 0.05%, \nand river channels only 0.025%. Rivers thus represent only one four-thousandth of the \nplanet's total water. Although rivers make up a small percen tage of the landscape, their \nimpact on the landscape extends far beyond the channel itself.   \n \nFrom a geomorphologic perspective, rivers have three main functions; to erode, \ntransport, and deposit sediment. In the proc ess of accomplishing these three tasks, rivers \ndestroy old landforms and create new ones. From a hydrological pers pective, rivers are \nan important component of the hydrological cy cle. Rivers are also  significant in a",
  "%. Rivers thus represent onl

In [12]:
# Define Tools for Vector Store Queries
def query_vector_store(vector_store, query: str) -> str:
    docs = vector_store.similarity_search(query, k=3)
    return " ".join([doc.page_content for doc in docs])

query_vector_1 = Tool(
    name="Query PDF 1",
    func=lambda query: query_vector_store(vector_store_1, query),
    description="Search PDF 1 vector store for answers."
)

query_vector_2 = Tool(
    name="Query PDF 2",
    func=lambda query: query_vector_store(vector_store_2, query),
    description="Search PDF 2 vector store for answers."
)

query_vector_3 = Tool(
    name="Query PDF 3",
    func=lambda query: query_vector_store(vector_store_3, query),
    description="Search PDF 3 vector store for answers."
)

In [41]:

from langchain.chat_models import ChatOpenAI


# No openai free credits :) using the local model

def get_llm():

    return ChatOpenAI(
    model="mistralai/Pixtral-12B-2409",
    temperature=0,
    max_retries=2,
    api_key="...",
    base_url="http://0.0.0.0:9109/v1",
)
get_llm().invoke("Hello, how are you?")

/tmp/ipykernel_932976/2560089988.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  return ChatOpenAI(


AIMessage(content="Hello! I'm functioning well, thank you. I'm here to help. How about you? How are you doing today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 9, 'total_tokens': 36, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Pixtral-12B-2409', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5e5ccc67-5967-42c3-8f17-39fde23828b7-0')

In [1]:
import json

# Path to the final.json file
file_path = "final.json"

# Read the JSON file
with open(file_path, "r") as file:
    keywords_data = json.load(file)

# Print the keywords
insurance_keys = keywords_data['keywords']["insurance"]
rivers_keys = keywords_data['keywords']["rivers"]
wildlife_keys = keywords_data['keywords']["wildlife"]
print(insurance_keys)
print(rivers_keys)
print(wildlife_keys)


['MODULE', '3', 'Practice', 'Life', 'Insurance', 'Notes', 'Claims', 'and', 'Settlement', 'DIPLOMA', 'INSURANCE', 'SERVICES', 'INTRODUCTION', 'OBJECTIVES', 'CLAIM', 'SETTLEMENT', 'Payment', 'claim', 'life', 'insurance', 'premium', 'Survival', 'Benefit', 'Claim', 'Maturity', 'Benfit', 'Claim', 'Death', 'Benefit', 'Claim', 'procedure', 'documents', 'forms', 'Insurance', 'policy', 'original', 'policy', 'document', 'age', 'proof', 'intimation', 'death', 'nominee', 'received', 'claim', 'nomination', 'policy', 'life', 'assured', 'disappeared', 'premature', 'death', 'claim', 'Indian', 'Evidence', 'Act', 'probate', 'succession', 'certificate', 'administrator', 'general', 'death', 'occurred', 'claim', 'concession', 'ex', 'gratia', 'claim', 'Ex', 'gratia', 'claims', 'options', 'IRDA', 'REGULATION', 'POLICYHOLDERS', '', 'PROCEDURE', 'CLAIM', 'Settlement', 'regulations', 'policyholder', 'interests', 'claim', 'paid', 'reasons', 'relevant', 'papers', 'interest', 'bank', 'rate', 'SUMMARY', 'terminal',

In [62]:
from langchain_core.prompts import ChatPromptTemplate


# Prompt
system = f"""You are an expert at routing a user question to the appropriate data source(s).

Based the user's question, you need to route the question to the appropriate data source (s). The data sources are:
["insurance"] for insurance-related queries
    if the question contains any of the following keywords: {insurance_keys}
["river"] for rivers-related queries
    if the question contains any of the following keywords: {rivers_keys}
["wildlife"] for wildlife-related queries
    if the question contains any of the following keywords: {wildlife_keys}
If there are multiple data sources, you should respond.
    ["insurance", "rivers"]

If the question does not contain any of the keywords, you should not route the question to any data source.
Respond: 
    ["insurance", "rivers", "wildlife"] 

Dont mention any other text in response execpt the list of data sources
"""


prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{question}"),
])

llm = get_llm()

# Define router
router = prompt | llm

question = """
What is the process for settlement of maturity claims?
"""

result = router.invoke({"question": question})
print(result)

content='["insurance"]' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 1000, 'total_tokens': 1005, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Pixtral-12B-2409', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-10b2ca93-211d-410c-bfbe-c4d5d7d85282-0'


In [ ]:
agents = result.content
print(agents)
agent_list = json.loads(agents)
print(agent_list)


["insurance"]
['insurance']


list

In [73]:

def pdf_agent(query: str, db_name: str):
    """Handles retrieval from ChromaDB and passes relevant chunks to LLM."""
    collection = chroma_client.get_or_create_collection(db_name,embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2"))

    results = collection.query(query_texts=[query], n_results=5, include=["documents"])   
    
    return results['documents']

retrieved_texts = []
for agent in agent_list:
    retrieved_texts.append(pdf_agent(question, agent))


In [76]:
def aggregator_agent(retrieved_texts: List[str], query: str):
    """Aggregates the retrieved texts and passes them to LLM."""

    print(retrieved_texts)
    response = query_llm_rest_pix([
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Answer based on the following :        {retrieved_texts}\n        Question: {query}"}
    ])

    return response

aggregator_agent(retrieved_texts,question)



[[['If the maturity claim is demanded within one year, before the\nmaturity it is called a discounted maturity claim. This amountis much less than the maturity claim.\n7.2.3 Death ClaimIf the life assured dies during the term of the policy, the death\nclaim arises. If the death has taken place within the first twoyears of the commencement of the policy, it is called an earlydeath claim and if the death has taken after 2 years, it iscalled a non early death claim.\nINTEXT QUESTIONS 7.1\n1. When the maturity claim is payable?\n2. When the death claim is payable?\n7.3 CLAIM DOCUMENTS & FORMS AND SETTLEMENT\nPROCEDURE\nWe will discuss in this section the insurance documents\nnecessary at the time of the final payment. The final paymentmay relate to the maturity or death claim payment.\n7.3.1 The documents required for payment of maturity\nclaim :\n(i) Age proof, if age is not admitted.\n(ii)Original policy document for cancellation.', '2. What is succession certificate?\n7.4. PROCEDURE OF 

'The process for the settlement of maturity claims in life insurance involves several steps:\n\n1. **Policy Maturity**: The policyholder must live through the duration of the policy to become eligible for the maturity value.\n\n2. **Notification**: The insurer sends a claim discharge voucher to the policyholder well before the date of maturity. This voucher needs to be signed and witnessed.\n\n3. **Required Documents**: To receive the maturity value, the policyholder must submit:\n   - Age proof, if age was not admitted at the time of proposal.\n   - The original policy document, which will be canceled upon payment.\n\n4. **Verification**: If age was not admitted during the proposal stage, age proof must be submitted before the payment of the maturity value.\n\n5. **Payment**: Once all the necessary documents are submitted and verified, the insurer processes the payment of the maturity claim.\n\n6. **No Nomination Role**: Since the policyholder is alive, the nomination (if any) does no